In [ ]:
import pandas as pd
import numpy as np
import json
import os

In [ ]:
data_dir = "/data/codes/sb-apa/data/scoring/apa-type-12/"

In [ ]:
path = f"{data_dir}/train.json"
with open(path, "r") as f:
    data = json.load(f)

In [ ]:
path = f"{data_dir}/pred.csv"
df = pd.read_csv(path)
df.head(1)

In [ ]:
df["pred"] = df["pred"].apply(lambda row: np.array(list(map(float, row.split()))))
df["label"] = df["label"].apply(lambda row: np.array(list(map(float, row.split()))))

In [ ]:
df["diff"] = df["label"] - df["pred"]
df["diff"] = df["diff"].apply(lambda row: np.abs(row))

df.head(1)

In [ ]:
threshold = 2 * 30/100

df["valid"] = df["diff"].apply(lambda row: row[row<threshold].shape[0])
df["total"] = df["diff"].apply(lambda row: row.shape[0])
df["percent"] = df["valid"] / df["total"]
df["invalid"] = df["diff"].apply(lambda row: row[row>threshold].shape[0])

In [ ]:
print(df.shape)
df = df[(df.invalid < 2)|(df.percent > 0.9)]
print(df.shape)

In [ ]:
filtered_data = {}
for index in df.index:
    _id = str(df["id"][index])
    filtered_data[_id] = data[_id]
    

In [ ]:
path = f"{data_dir}/filtered-data.json"
with open(path, "w") as f:
    json_obj = json.dumps(filtered_data, indent=4, ensure_ascii=False)
    f.write(json_obj)